Looking at the Hopkins COVID-19 data that was cloned from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import statistics
from datetime import datetime as dt

df_confirmed = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

#df_deaths = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
#df_recovered = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [2]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,990,990,990,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,413,414,414,418,418,422,426,428,428,429
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,296,296,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,102,119,120,124,124,125


In [3]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 53 columns):
Province/State    240 non-null object
Country/Region    347 non-null object
Lat               347 non-null float64
Long              347 non-null float64
1/22/20           347 non-null int64
1/23/20           347 non-null int64
1/24/20           347 non-null int64
1/25/20           347 non-null int64
1/26/20           347 non-null int64
1/27/20           347 non-null int64
1/28/20           347 non-null int64
1/29/20           347 non-null int64
1/30/20           347 non-null int64
1/31/20           347 non-null int64
2/1/20            347 non-null int64
2/2/20            347 non-null int64
2/3/20            347 non-null int64
2/4/20            347 non-null int64
2/5/20            347 non-null int64
2/6/20            347 non-null int64
2/7/20            347 non-null int64
2/8/20            347 non-null int64
2/9/20            347 non-null int64
2/10/20           347 non-null int64
2

I think I just need to work with the confirmed DataFrame. It's set up such that each row is a specific location, and the columns are different dates.



In [4]:
#creating template for dataframes to record various values

df_template = df_confirmed.copy()


df_template.head(), df_template.size

(  Province/State  Country/Region      Lat      Long  1/22/20  1/23/20  \
 0          Anhui  Mainland China  31.8257  117.2264        1        9   
 1        Beijing  Mainland China  40.1824  116.4142       14       22   
 2      Chongqing  Mainland China  30.0572  107.8740        6        9   
 3         Fujian  Mainland China  26.0789  117.9874        1        5   
 4          Gansu  Mainland China  36.0611  103.8343        0        2   
 
    1/24/20  1/25/20  1/26/20  1/27/20  ...  3/1/20  3/2/20  3/3/20  3/4/20  \
 0       15       39       60       70  ...     990     990     990     990   
 1       36       41       68       80  ...     413     414     414     418   
 2       27       57       75      110  ...     576     576     576     576   
 3       10       18       35       59  ...     296     296     296     296   
 4        2        4        7       14  ...      91      91      91      91   
 
    3/5/20  3/6/20  3/7/20  3/8/20  3/9/20  3/10/20  
 0     990     990     9

In [5]:
# Creating template out of JHU table format

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

for x in df_template.columns.values:
    if (x not in do_not_include):
        for y in range(df_template['1/22/20'].values.size):
            df_template.at[y, x] = 0
            
            
df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Adding the rest of the beginning of January

for q in range(1, 22):
    day_str = str(q)
    date_str = '1/' + day_str + '/20'
    df_template[date_str] = 0    

df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 74 columns):
Province/State    240 non-null object
Country/Region    347 non-null object
Lat               347 non-null float64
Long              347 non-null float64
1/22/20           347 non-null int64
1/23/20           347 non-null int64
1/24/20           347 non-null int64
1/25/20           347 non-null int64
1/26/20           347 non-null int64
1/27/20           347 non-null int64
1/28/20           347 non-null int64
1/29/20           347 non-null int64
1/30/20           347 non-null int64
1/31/20           347 non-null int64
2/1/20            347 non-null int64
2/2/20            347 non-null int64
2/3/20            347 non-null int64
2/4/20            347 non-null int64
2/5/20            347 non-null int64
2/6/20            347 non-null int64
2/7/20            347 non-null int64
2/8/20            347 non-null int64
2/9/20            347 non-null int64
2/10/20           347 non-null int64
2

In [8]:
# For description of the different units being captured from the API,
# please see https://darksky.net/dev/docs
# Requests are being made in SI units

# dataframe to record median maximum temperature
df_tMax = df_template.copy()

# dataframe to record median minimum temperature
df_tMin = df_template.copy()

# dataframe to record median humidity
df_humidity = df_template.copy()

# dataframe to record median UV index
df_uvIndex = df_template.copy()

# dataframe to record median cloud cover
df_cloud = df_template.copy()

# dataframe to record weather summaries
#df_summaries = df_template.copy()

# dataframe to record precipitation probability
df_precipprob = df_template.copy()

# dataframe to record dewpoint
df_dewpoint = df_template.copy()

# dataframe to record pressure
df_pressure = df_template.copy()

# dataframe to record windspeed
df_windspeed = df_template.copy()

# dataframe to record "ozone" reading
df_ozone = df_template.copy()

# dataframe to record sunrise time
df_sunrise = df_template.copy()

# dataframe to record sunset time
df_sunset = df_template.copy()

In [24]:
#Darksky API key
API_KEY = 'API KEY'

#number of seconds in one day
one_day = 86400

t = '1583625600'

latitude = '31.8257'
longitude = '117.2264'

# sample get request 
#GET https://api.darksky.net/forecast/0123456789abcdef9876543210fedcba/
#42.3601,-71.0589,255657600?exclude=currently,flags

url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t

url = url + '?exclude=currently,flags&units=si'
url

'https://api.darksky.net/forecast/0dbf1141c37674b871ee3d76c9593f31/31.8257,117.2264,1583625600?exclude=currently,flags&units=si'

In [25]:
response = requests.get(url)

In [26]:
response.content

b'{"latitude":31.8257,"longitude":117.2264,"timezone":"Asia/Shanghai","hourly":{"summary":"Possible light rain throughout the day.","icon":"rain","data":[{"time":1583596800,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":11.47,"apparentTemperature":11.47,"dewPoint":8.95,"humidity":0.85,"pressure":1015.9,"windSpeed":2.76,"windGust":4.26,"windBearing":105,"cloudCover":1,"uvIndex":0,"visibility":16.093,"ozone":326.1},{"time":1583600400,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":10.94,"apparentTemperature":10.94,"dewPoint":8.72,"humidity":0.86,"pressure":1015.4,"windSpeed":2.86,"windGust":5.08,"windBearing":100,"cloudCover":1,"uvIndex":0,"visibility":16.093,"ozone":324},{"time":1583604000,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":10.54,"apparentTemperature":10.54,"dewPoint":8.48,"humidity":0.87,"pressure":1014.6,"windSpeed":3.12,"windGust":6.05,"

In [27]:
info = json.loads(response.content)

info['daily']['data'][0]

{'time': 1583596800,
 'summary': 'Light rain throughout the day.',
 'icon': 'rain',
 'sunriseTime': 1583620260,
 'sunsetTime': 1583662500,
 'moonPhase': 0.45,
 'precipIntensity': 0.1234,
 'precipIntensityMax': 0.6269,
 'precipIntensityMaxTime': 1583636580,
 'precipProbability': 0.84,
 'precipType': 'rain',
 'temperatureHigh': 15.24,
 'temperatureHighTime': 1583653620,
 'temperatureLow': 10.28,
 'temperatureLowTime': 1583712000,
 'apparentTemperatureHigh': 14.96,
 'apparentTemperatureHighTime': 1583653620,
 'apparentTemperatureLow': 10.55,
 'apparentTemperatureLowTime': 1583712000,
 'dewPoint': 9.72,
 'humidity': 0.83,
 'pressure': 1012.1,
 'windSpeed': 5.19,
 'windGust': 12.34,
 'windGustTime': 1583624700,
 'windBearing': 98,
 'cloudCover': 0.61,
 'uvIndex': 6,
 'uvIndexTime': 1583639940,
 'visibility': 15.901,
 'ozone': 319.4,
 'temperatureMin': 9.14,
 'temperatureMinTime': 1583611200,
 'temperatureMax': 15.24,
 'temperatureMaxTime': 1583653620,
 'apparentTemperatureMin': 7.1,
 'appar

In [22]:
df_tMax.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# pull data from darksky weather API

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'API KEY'

# Number of seconds in one day
one_day = 86400


# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
                
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset

          
            # API has a daily limit, so we're just taking a break here
            if (y == 10):
                break
                
    if (x == '1/22/20'):
        break


In [32]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,Anhui,Mainland China,31.82570,117.2264,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,24,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# test run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'API KEY'

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            #url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            #url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            #response = requests.get(url)
                
            # Putting the API response into the JSON thing
            #info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            #easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset
            
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
                
    #if (x == '1/22/20'):
     #   break


In [38]:
df_tMax.tail(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
243,"Marion County, OR",US,44.8446,-122.5927,486,486,486,486,486,486,...,486,486,486,486,486,486,486,486,486,486
244,"Okaloosa County, FL",US,30.5773,-86.6611,488,488,488,488,488,488,...,488,488,488,488,488,488,488,488,488,488
245,"Polk County, GA",US,34.0132,-85.1479,490,490,490,490,490,490,...,490,490,490,490,490,490,490,490,490,490
246,"Riverside County, CA",US,33.9533,-117.3961,492,492,492,492,492,492,...,492,492,492,492,492,492,492,492,492,492
247,"Shelby County, TN",US,35.1269,-89.9253,494,494,494,494,494,494,...,494,494,494,494,494,494,494,494,494,494
248,"Spokane County, WA",US,47.6587,-117.4225,496,496,496,496,496,496,...,496,496,496,496,496,496,496,496,496,496
249,"St. Louis County, MO",US,38.6103,-90.4125,498,498,498,498,498,498,...,498,498,498,498,498,498,498,498,498,498
250,"Suffolk County, NY",US,40.9849,-72.6151,500,500,500,500,500,500,...,500,500,500,500,500,500,500,500,500,500
251,"Ulster County, NY",US,41.8586,-74.3118,502,502,502,502,502,502,...,502,502,502,502,502,502,502,502,502,502
252,"Unassigned Location, VT",US,44.3378,-72.7563,504,504,504,504,504,504,...,504,504,504,504,504,504,504,504,504,504


In [10]:
# The real API run

# Columns to be skipped when iterating through the DataFrame
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = '4c7a49e1f2860b69e0ebc40af6d194f8'

# Setting a counter for API calls as an emergency break, in case the thing gets out of control
counter = 0

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        t = str(t)
                
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
                      
            # latitude and longitude coordinates of the row to be passed to the API            
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
                        
            # Building the URL for the API get
            url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t
            url = url + '?exclude=currently,flags&units=si'
                
            # Getting the API call
            response = requests.get(url)
            # Incrementing the counter
            counter = counter + 1
                
            # Putting the API response into the JSON thing
            info = json.loads(response.content)
            # Making a variable to more easily acccess JSON response data
            easy_info = info['daily']['data'][0]
                
            # Reading the JSON data
            tMax = easy_info['temperatureHigh']
            tMin = easy_info['temperatureLow']
            hum = easy_info['humidity'] * 100
            uvee = easy_info['uvIndex']
            clouds = easy_info['cloudCover'] * 100
            precip = easy_info['precipProbability'] * 100
            dew = easy_info['dewPoint']
            pressure = easy_info['pressure']
            wind = easy_info['windSpeed']
            ozone = easy_info['ozone']
            sunrise = easy_info['sunriseTime']
            sunset = easy_info['sunsetTime']
            
            # Recording the data into the respective dataframes
            df_tMax.at[y, x] = tMax
            df_tMin.at[y, x] = tMin
            df_humidity.at[y, x] = hum
            df_uvIndex.at[y, x] = uvee
            df_cloud.at[y, x] = clouds
            df_precipprob.at[y, x] = precip
            df_dewpoint.at[y, x] = dew
            df_pressure.at[y, x] = pressure
            df_windspeed.at[y, x] = wind
            df_ozone.at[y, x] = ozone
            df_sunrise.at[y,x] = sunrise
            df_sunset.at[y,x] = sunset
            
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
                
    # By my calculation, this should run 17340 times, so I set the counter break at that number + 10, for a little buffer
    # in case I've screwed something up
    #if (counter == 17350):
     #   break
     #   break


In [12]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_tMin.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_humidity.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,80,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,79,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,60,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,82,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_uvIndex.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_cloud.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,67,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_precipprob.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_dewpoint.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_pressure.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,1026,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1021,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1015,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_windspeed.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_ozone.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,278,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,356,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,271,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,253,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,322,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,246,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,246,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,251,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,237,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,341,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# writing CSVs of what I've got so far

df_tMax.to_csv('./csv/tMax.csv', index=False)

df_tMin.to_csv('./csv/tMin.csv', index=False)

df_humidity.to_csv('./csv/humidity.csv', index=False)

df_uvIndex.to_csv('./csv/uv.csv', index=False)

df_cloud.to_csv('./csv/cloud.csv', index=False)

df_precipprob.to_csv('./csv/precip.csv', index=False)

df_dewpoint.to_csv('./csv/dew.csv', index=False)

df_pressure.to_csv('./csv/pressure.csv', index=False)

df_windspeed.to_csv('./csv/wind.csv', index=False)

df_ozone.to_csv('./csv/ozone.csv', index=False)

df_sunrise.to_csv('./csv/sunrise.csv', index=False)

df_sunset.to_csv('./csv/sunset.csv', index=False)
